In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import xgboost as xg
import pickle
import matplotlib.pyplot as plt

In [2]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test

In [3]:
def r2_prediction(regressor,X_test,y_test):
     y_pred = regressor.predict(X_test)
     from sklearn.metrics import r2_score
     r2=r2_score(y_test,y_pred)
     return r2

In [4]:
def Linear(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [5]:
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'linear')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [6]:
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [7]:
def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training setC
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [8]:
def random(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2

In [9]:
def xgboost(X_train, y_train, X_test):
    from xgboost import XGBRegressor
    regressor = XGBRegressor(n_estimators=100, random_state=0)
    regressor.fit(X_train, y_train)
    r2 = r2_prediction(regressor, X_test, y_test)
    return r2

In [10]:
def rfeFeature(indep_X,dep_Y,n):
        rfelist=[]
        
        from sklearn.linear_model import LinearRegression
        lin = LinearRegression()
        
        from sklearn.svm import SVR
        SVRl = SVR(kernel = 'linear')
        
        from sklearn.svm import SVR
        #SVRnl = SVR(kernel = 'rbf')
        
        from sklearn.tree import DecisionTreeRegressor
        dec = DecisionTreeRegressor(random_state = 0)
        
        from sklearn.ensemble import RandomForestRegressor
        rf = RandomForestRegressor(n_estimators = 10, random_state = 0)

        from xgboost import XGBRegressor
        xgb = XGBRegressor(n_estimators=10, random_state=0)
        
        
        rfemodellist=[lin,SVRl,dec,rf,xgb] 
        for i in   rfemodellist:
            print(i)
            log_rfe = RFE(i, n_features_to_select=n)
            log_fit = log_rfe.fit(indep_X, dep_Y)
            log_rfe_feature=log_fit.transform(indep_X)
            rfelist.append(log_rfe_feature)
            selected_columns = indep_X.columns[log_rfe.get_support()]
        return rfelist, selected_columns

In [11]:
dataset1=pd.read_csv("Concrete_Data.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

In [12]:
indep_X=df2.drop('csMPa', axis=1)
dep_Y=df2['csMPa']

In [37]:
rfelist, selected_columns = rfeFeature(indep_X, dep_Y, 5)
   
# Use only the top-selected features
X_selected = indep_X[selected_columns]

# Split and scale and top selected variable is given as parameter
X_train, X_test, y_train, y_test = split_scalar(X_selected, dep_Y)

acclin = []
accsvml = []
accdes = []
accrf = []
accxgb = []

r2_lin = Linear(X_train, y_train, X_test)
r2_sl = svm_linear(X_train, y_train, X_test)
r2_d = Decision(X_train, y_train, X_test)
r2_r = random(X_train, y_train, X_test)
r2_xgb = xgboost(X_train, y_train, X_test)

#Append values for one row
acclin.append(r2_lin)
accsvml.append(r2_sl)
accdes.append(r2_d)
accrf.append(r2_r)
accxgb.append(r2_xgb)

#Build result DataFrame (same row repeated)
model_names = ['Linear', 'SVC', 'Random', 'DecisionTree', 'XGB']
result_df = pd.DataFrame(index=model_names, columns=['Linear', 'SVMl', 'Decision', 'Random', 'XGB'])

# Fill all rows with same values
for model in result_df.index:
    result_df.loc[model] = [r2_lin, r2_sl, r2_d, r2_r, r2_xgb]


LinearRegression()
SVR(kernel='linear')
DecisionTreeRegressor(random_state=0)
RandomForestRegressor(n_estimators=10, random_state=0)
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=10,
             n_jobs=None, num_parallel_tree=None, ...)


In [25]:
#4
print(result_df)
print("\nSelected Columns:", selected_columns.tolist())

                Linear      SVMl  Decision   Random       XGB
Linear        0.540786  0.532958  0.699228  0.84169  0.896804
SVC           0.540786  0.532958  0.699228  0.84169  0.896804
Random        0.540786  0.532958  0.699228  0.84169  0.896804
DecisionTree  0.540786  0.532958  0.699228  0.84169  0.896804
XGB           0.540786  0.532958  0.699228  0.84169  0.896804

Selected Columns: ['cement', 'water', 'superplasticizer', 'age']


In [38]:
#5
print(result_df)
print("\nSelected Columns:", selected_columns.tolist())

               Linear      SVMl  Decision    Random      XGB
Linear        0.60569  0.595759  0.818562  0.897536  0.91538
SVC           0.60569  0.595759  0.818562  0.897536  0.91538
Random        0.60569  0.595759  0.818562  0.897536  0.91538
DecisionTree  0.60569  0.595759  0.818562  0.897536  0.91538
XGB           0.60569  0.595759  0.818562  0.897536  0.91538

Selected Columns: ['cement', 'slag', 'water', 'superplasticizer', 'age']


In [19]:
#6
print(result_df)
print("\nSelected Columns:", selected_columns.tolist())

               Linear     SVMl  Decision   Random       XGB
Linear        0.60698  0.59294  0.832286  0.89348  0.917574
SVC           0.60698  0.59294  0.832286  0.89348  0.917574
Random        0.60698  0.59294  0.832286  0.89348  0.917574
DecisionTree  0.60698  0.59294  0.832286  0.89348  0.917574
XGB           0.60698  0.59294  0.832286  0.89348  0.917574

Selected Columns: ['cement', 'slag', 'water', 'superplasticizer', 'fineaggregate', 'age']


In [21]:
#7
print(result_df)
print("\nSelected Columns:", selected_columns.tolist())

               Linear      SVMl  Decision    Random       XGB
Linear        0.62755  0.611903  0.780093  0.891062  0.924075
SVC           0.62755  0.611903  0.780093  0.891062  0.924075
Random        0.62755  0.611903  0.780093  0.891062  0.924075
DecisionTree  0.62755  0.611903  0.780093  0.891062  0.924075
XGB           0.62755  0.611903  0.780093  0.891062  0.924075

Selected Columns: ['cement', 'slag', 'flyash', 'water', 'superplasticizer', 'fineaggregate', 'age']


In [23]:
#8
print(result_df)
print("\nSelected Columns:", selected_columns.tolist())

               Linear     SVMl  Decision    Random       XGB
Linear        0.62347  0.61833  0.806542  0.877116  0.911823
SVC           0.62347  0.61833  0.806542  0.877116  0.911823
Random        0.62347  0.61833  0.806542  0.877116  0.911823
DecisionTree  0.62347  0.61833  0.806542  0.877116  0.911823
XGB           0.62347  0.61833  0.806542  0.877116  0.911823

Selected Columns: ['cement', 'slag', 'flyash', 'water', 'superplasticizer', 'coarseaggregate', 'fineaggregate', 'age']


### Totally we have 8 Features and we tried from 4 to 8 and the overall good performance came from 5 to 8 features
### So am selecting 5 features and selecting XGB regression model

In [39]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
param_grid={'n_estimators':[10,50,100,150],'learning_rate': [0.01, 0.1,0.2,0.3],'max_depth': [3, 5, 7],'subsample': [0.8, 1.0],
            'booster':['gbtree', 'dart'],'colsample_bytree': [0.8, 1.0]}
grid=GridSearchCV(XGBRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    lear...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'booster': ['gbtree', 'dart'],
                         'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2, 0.3],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [10, 50, 100, 150],
                         'subsample': [0.8, 1.0]},
             verbose=3)

In [40]:
re=grid.cv_results_
grid_predictions=grid.predict(X_test)

rx_sc=r2_score(y_test,grid_predictions)
print("The R_score value for best parameter:{}".format(grid.best_params_),rx_sc)

The R_score value for best parameter:{'booster': 'gbtree', 'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 150, 'subsample': 0.8} 0.9225221712656226


In [41]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_booster,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.019161,0.006045,0.000507,0.001013,gbtree,0.8,0.01,3,10,0.8,"{'booster': 'gbtree', 'colsample_bytree': 0.8,...",0.074099,0.093979,0.098358,0.111729,0.108824,0.097398,0.013352,383
1,0.014777,0.003992,0.000403,0.000494,gbtree,0.8,0.01,3,10,1.0,"{'booster': 'gbtree', 'colsample_bytree': 0.8,...",0.072051,0.097012,0.100742,0.115031,0.109148,0.098797,0.014785,381
2,0.033729,0.004828,0.000000,0.000000,gbtree,0.8,0.01,3,50,0.8,"{'booster': 'gbtree', 'colsample_bytree': 0.8,...",0.359428,0.383123,0.364368,0.402751,0.385974,0.379129,0.015664,357
3,0.032779,0.003444,0.002822,0.003733,gbtree,0.8,0.01,3,50,1.0,"{'booster': 'gbtree', 'colsample_bytree': 0.8,...",0.346959,0.389231,0.359544,0.402124,0.386778,0.376927,0.020413,359
4,0.059326,0.002490,0.002111,0.003307,gbtree,0.8,0.01,3,100,0.8,"{'booster': 'gbtree', 'colsample_bytree': 0.8,...",0.571982,0.591258,0.556760,0.609784,0.585997,0.583157,0.017913,333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,0.240623,0.008178,0.004732,0.003401,dart,1.0,0.3,7,50,1.0,"{'booster': 'dart', 'colsample_bytree': 1.0, '...",0.890019,0.929728,0.912695,0.887469,0.867089,0.897400,0.021684,165
380,0.757552,0.015217,0.010172,0.000194,dart,1.0,0.3,7,100,0.8,"{'booster': 'dart', 'colsample_bytree': 1.0, '...",0.902485,0.937058,0.929956,0.888064,0.846117,0.900736,0.032619,135
381,0.725565,0.025035,0.009724,0.001275,dart,1.0,0.3,7,100,1.0,"{'booster': 'dart', 'colsample_bytree': 1.0, '...",0.890193,0.930941,0.914750,0.888899,0.867260,0.898409,0.022152,156
382,1.548623,0.021256,0.009365,0.004241,dart,1.0,0.3,7,150,0.8,"{'booster': 'dart', 'colsample_bytree': 1.0, '...",0.903143,0.929164,0.926200,0.888075,0.843809,0.898078,0.031074,160


In [42]:
cement_input=float(input("Cement:"))
slag_input=float(input("Slag:"))
water_input=float(input("Water:"))
superplasticizer_input=float(input("Superplasticizer:"))
age_input=int(input("Age:"))

Cement: 540
Slag: 148.5
Water: 280
Superplasticizer: 2.6
Age: 28


In [43]:
Future_Prediction=grid.predict([[cement_input,slag_input,water_input,superplasticizer_input,age_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[65.94959]
